# Digit Recognition
digitrec.py is a Python script that takes an image file containing a handwritten digit and identifies the digit using a supervised learning algorithm. This will be done using the MNIST dataset to train and test the neural network. This script uses [keras](https://github.com/keras-team/keras), a high level neural network API. 

### Digit Dataset
First things first, we need to open and read the MNIST dataset. This has already been covered in the [MNIST dataset notebook](https://github.com/HUGHBRADY/Emerging_Tech/blob/master/MNIST%20Dataset/mnist-dataset.ipynb) in this repository. 

### Imports & MNIST set up

In [1]:
import numpy as np
import keras as kr
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import gzip

# Open and read all of the MNIST files
with gzip.open('data/t10k-images-idx3-ubyte.gz', 'rb') as f:  
    raw_test_img = f.read()
with gzip.open('data/t10k-labels-idx1-ubyte.gz', 'rb') as f:  
    raw_test_lbl = f.read()
with gzip.open('data/train-images-idx3-ubyte.gz', 'rb') as f:  
    raw_train_img = f.read()
with gzip.open('data/train-labels-idx1-ubyte.gz', 'rb') as f:  
    raw_train_lbl = f.read()

# Next we'll trimodelthe excess bytes at the start and convert the bytes to integers
test_img = np.frombuffer(raw_test_img, dtype = np.uint8, offset = 16) / 255
test_lbl = np.frombuffer(raw_test_lbl, dtype = np.uint8, offset = 8)
train_img = np.frombuffer(raw_train_img, dtype = np.uint8, offset = 16) / 255
train_lbl = np.frombuffer(raw_train_lbl, dtype = np.uint8, offset = 8)

# Finally we reshape the image arrays
test_img  =  test_img.reshape(10000, 784)
train_img = train_img.reshape(60000, 784)

# Convert class vectors to binary class matrices 
train_lbl = kr.utils.to_categorical(train_lbl, 10)
test_lbl = kr.utils.to_categorical(test_lbl, 10)

C:\Users\Hughballs\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Keras 
We also need to set up our neural network. Keras' core data structure is a model, which is a way to organize layers. The simpliest of these models is the Sequential model so it's the perfect one to start learning with. 

### Building the Model
The Sequential model is a linear stack of layers. The most important layers are the first and last, or input and output layers. The input layer defines the shape of the input. 
There is a dense connection between the neurons meaning that every neuron in every layer is connected to every neuron in the neighbouring layers.
The output layer has ten neurons that correspond to the possible answers (0 - 9).

In [2]:
# Create our keras model
model = kr.models.Sequential()

# Layers to add to our sequential model. The first layer defines the input shape
# Dropout randomly selects nodes to drop, resulting in a network capable of better generalization
model.add(Dense(1000, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))                                                 
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              785000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                10010     
Total params: 1,796,010
Trainable params: 1,796,010
Non-trainable params: 0
_________________________________________________________________


Once these layers are added we must compile the neural network.

In [3]:
# Defines the loss function, optimizer and metrics and is needed for training
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

### Training the Model
Once the model has been built and the images have been read in, we have to train the model to recognize the images. The training set of 60000 images are used and passed to the networks first layer of 784 neurons. The training images are sent as input and the training labels are attached as the expected output. Epochs refer to the amount of times the input will be processed, which I've set to 20. Finally the batch size in this case refers the amount of images that will be sent into the neural network at once.

In [4]:
model.fit(train_img, train_lbl, batch_size=100, epochs=20, validation_data=(test_img, test_lbl))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 22s 362us/step - loss: 0.2157 - acc: 0.9346 - val_loss: 0.0930 - val_acc: 0.9707
Epoch 2/20
60000/60000 [==============================] - 21s 349us/step - loss: 0.0987 - acc: 0.9714 - val_loss: 0.0863 - val_acc: 0.9758
Epoch 3/20
60000/60000 [==============================] - 20s 336us/step - loss: 0.0752 - acc: 0.9791 - val_loss: 0.0922 - val_acc: 0.9776
Epoch 4/20
60000/60000 [==============================] - 21s 342us/step - loss: 0.0682 - acc: 0.9826 - val_loss: 0.0830 - val_acc: 0.9819
Epoch 5/20
60000/60000 [==============================] - 20s 336us/step - loss: 0.0585 - acc: 0.9845 - val_loss: 0.0858 - val_acc: 0.9815
Epoch 6/20
60000/60000 [==============================] - 20s 334us/step - loss: 0.0553 - acc: 0.9859 - val_loss: 0.0906 - val_acc: 0.9811
Epoch 7/20
60000/60000 [==============================] - 21s 348us/step - loss: 0.0513 - acc: 0.9875 - val_loss: 0.

### Output
Return the results of the loses and the accuracy of the mnist neural network.

In [6]:
score = model.evaluate(test_img, test_lbl, verbose=0)

# Finally, print the loss and the accuracy
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.14759161900484996
Test accuracy: 0.9807
